#### ПАРСИНГ ИНФОРМАЦИИ ПО ГОРОДАМ

Не стоит надеяться, что этот ноутбук будет работать. Конкретно в этом случае парсинг довольно нетривиальная (по крайней мере для меня) задача, поэтому многие вещи приходилось править практически в ручном режиме:

In [2]:
import pandas as pd
import numpy as np
import requests
import wikipedia as wp
import re

In [ ]:
def keyword_mask(data_, column_, list_, operator='or'):  
    if operator == 'or':
        mask_full = False
        for prop in list_:
            mask_ = data_[column_].str.find(prop) + 1
            mask_ = mask_.astype(bool)
            mask_full = mask_full | mask_
        return mask_full
    elif operator == 'and':
        mask_full = True
        for prop in list_:
            mask_ = data_[column_].str.find(prop) + 1
            mask_ = mask_.astype(bool)
            mask_full = mask_full & mask_
        return mask_full

In [138]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [288]:
data_zip = pd.read_csv('data/zipcode_parser_data.csv', index_col='Unnamed: 0', dtype='object')

Спарсим корректные названия городов с помощью индекса. Испольуем [этот API](https://api-ninjas.com/):

In [172]:
zip_list = data_zip['zipcode'].unique().tolist()
city_list = {}
city_error_list = {}
total = len(zip_list)

# for i, zip_ in enumerate(zip_list):   
#     try:      
#         api_url = 'https://api.api-ninjas.com/v1/zipcode?zip={}'.format(zip_)
#         response = requests.get(api_url, headers={'X-Api-Key': '/JkP/JwK9iI5nk/fwSXseQ==7qi1yzX4NLWV561W'})
#         city = response.json()[0]['city']
#         city_list[zip_] = city
#         print(f'Zipcode {zip_} for {city} resolved succefully! ({total - i} zipcodes to go!)', end='\r')
#     except (KeyError, IndexError) as error:
#         print(f'An issue occured with zipcode {zip_}')
#         city_error_list[zip_] = error

In [103]:
parsed_list = list(city_list.keys())
parsed_list = [str(el) for el in parsed_list]
zip_list = [el for el in zip_list if el not in parsed_list]

In [74]:
parsed_error_list = list(city_error_list.keys())
parsed_error_list = [str(el) for el in parsed_error_list]
zip_list = [el for el in zip_list if el not in parsed_error_list]

In [289]:
data_zip['city_correct'] = data_zip['zipcode'].map(city_list)

Проверка адреса в википедии на валидность:

In [287]:
# us_state_to_abbrev_swapped = {v: k for k, v in us_state_to_abbrev.items()}
# data_zip['state_full_name'] = data_zip['state'].map(us_state_to_abbrev_swapped)
# data_zip['city+state'] = data_zip['city_correct'] + ', ' + data_zip['state_full_name']
# cities = data_zip['city+state'].unique().tolist()
# cities = pd.DataFrame(cities, columns=['city'])
# cities['city'] = cities['city'].str.replace(' ', '_')

# city_wiki_url = {}
# city_url_response = {}
# total = cities.shape[0]

# for i, city in enumerate(cities['city'].unique().tolist()):
#     url = 'https://en.wikipedia.org/wiki/' + city
#     city_wiki_url[city] = url
#     response = requests.get(url)
#     city_url_response[city] = response.status_code
#     if code == 200:
#         print(f'City: {city}, code={response.status_code}, {total - i} more to go', end='\r')
#     else:
#         print(f'Issue with city {city}, code={response.status_code}')

Пример парсинга таблицы с википедии:

In [ ]:
# парсинг таблиц с википедии

# html = wp.page(city).html()
# try: 
#     df_largest_cities = pd.read_html(html)[0]
# except IndexError:
#     df_largest_cities = pd.read_html(html)[1]

Используем данные таблицы с [этого сайта](https://simplemaps.com/data/us-cities):

In [311]:
data_zip = data_zip.drop('city', axis=1)
data_zip = data_zip.reset_index()
data_zip = data_zip.rename(columns={'city_correct':'city'})

cities_info = pd.read_csv('data/us_cities.csv')
cities_info = cities_info[['city', 'lat', 'lng', 'population', 'density', 'state_id']]

data_info = data_zip.merge(cities_info, how='left', left_on=['city', 'state'], right_on=['city', 'state_id'])
data_info = data_info.drop(index=13334, axis=0)

data_info = data_info.drop('state_id', axis=1)

Парсим недостающую информацию по городам:

In [359]:
# mask_nan = data_info.lat.isna()

# city_list_nan = data_info.city[mask_nan].unique().tolist()
# city_data = {}
# city_data_errors = {}

# total = len(city_list_nan)

# for i, city in enumerate(city_list_nan):   
#     try:      
#         api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(city)
#         response = requests.get(api_url, headers={'X-Api-Key': 'yrRFg510NuWDi9uzvWt85Q==Vr7GuJDVE0PjbsD2'})
#         city_resp = response.json()
#         city_data[city] = city_resp
#         print(f'{city} info parsed succefully! ({total - i} cities to go!)', end='\r')
#     except (KeyError, IndexError) as error:
#         print(f'An issue occured with city {city}')
#         city_data_errors[city] = error

Вставляем значения координат и населения:

In [403]:
# non_us_city = []
# idx_error_city = []
# for city in city_data.keys():
#     try:
#         if city_data[city][0]['country'] == 'US':
#             data_info.loc[data_info.city == city, 'lat'] = city_data[city][0]['latitude']
#             data_info.loc[data_info.city == city, 'lng'] = city_data[city][0]['longitude']
#             data_info.loc[data_info.city == city, 'population'] = city_data[city][0]['population']
#         else:
#             non_us_city.append(city)
#     except IndexError as e:
#         idx_error_city.append(city)

Спарсим аннотацию, таблицы и координаты с википедии:

In [ ]:
# mask_nan = data_info.lat.isna()
# city_list_nan = data_info.loc[mask_nan, 'city+state'].unique().tolist()
# total = len(city_list_nan)

# city_coords = {}
# city_df = {}
# city_error = {}

# for i, city in enumerate(city_list_nan):
#     url = 'https://en.wikipedia.org/wiki/' + city
#     response = requests.get(url)
#     city_url_response[city] = response.status_code
#     if code == 200:
#         try:
#             page = wp.page(city)
#         except (wp.PageError, wp.DisambiguationError) as error:
#             city_error[city] = error
#             print(f'Issue with city {city}, code={error}')
#         try: 
#             city_df[city] = pd.read_html(page.html())[0]
#         except IndexError:
#             city_df[city] = pd.read_html(page.html())[1]
#         city_coords[city] = page.coordinates
#         print(f'City: {city}, code={code}, {total - i} more to go', end='\r')
#     else:
#         print(f'Issue with city {city}, code={code}')
#         city_error[city] = code

In [497]:
# city_coords = {}
# city_info = {}
# city_error = {}
# city_parsed_list = []

# for city in city_total_errors_list_state:
#     try:
#         page = wp.page(city)
#         city_coords[city] = page.coordinates
#         city_info[city] = page.summary
#         city_parsed_list.append(city)
#         print(f'City {city} info parsed!', end='\r')
#     except (wp.PageError, KeyError) as e:
#         city_error[city] = e
#         print(f'Issue with {city} city!')

Issue with Stow, NY city!arsed!!
Issue with Arrington, TN city!!sed!
Issue with Brant, NY city!
Issue with Rowe, VA city!
Issue with Camby, IN city!ed!parsed!d!ed!
Issue with Playa Vista, CA city!!!!sed!d!
Issue with Mc Leansville, NC city!sed!!
Issue with Busy, KY city!arsed! parsed!
Issue with Drift, KY city! parsed!
Issue with Ceresco, MI city!parsed!sed!
Issue with Graysville, GA city!!parsed!
Issue with Mallie, KY city!d!d!parsed!d!


Запишем координаты:

In [550]:
# data_info['city_2'] = data_info['city'] + ', ' + data_info['state']

# for city in city_coords.keys():
#     data_info.loc[data_info.city_2 == city, 'lat'] = float(city_coords[city][0])
#     data_info.loc[data_info.city_2 == city, 'lng'] = float(city_coords[city][1])

#### ПРАВКА ТАБЛИЦЫ ПО ЛОКАЛЬНЫМ КООРДИНАТАМ

Удалим города, которые упоминаются менее 10 раз:

In [151]:
# найдём количество строк для конкретного города в датасете

unique_cities = data_info.loc[data_info.isna().density, 'city_2'].unique().tolist()
cities_q = {}

for city in unique_cities:
    q = data_info[data_info.city_2 == city].shape[0]
    if q not in cities_q.keys():
        cities_q[q] = []
        cities_q[q].append(city)
    else:
        cities_q[q].append(city) 

In [529]:
cities_to_remove = []
for k in range(1, 10):
    cities_to_remove.extend(cities_q[k])

In [538]:
def filter_rows_by_values(data, col, values):
    return data[~data[col].isin(values)]

data_info = filter_rows_by_values(data_info, 'city', cities_to_remove)

In [76]:
city_total_errors_list = []
city_total_errors_list_state = []

for city in data_info.city[data_info.density.isna()].unique().tolist():
    city_total_errors_list.append(city)

for city in data_info.city_2[data_info.density.isna()].unique().tolist():
    city_total_errors_list_state.append(city)

In [ ]:
# замена в списках слова Saint - иначе парсер не находит город, или находит, но не тот

# city_saint = []
# for el in city_total_errors_list:
#     if el.find('Saint') != -1:
#         city_saint.append(el.replace('Saint ', 'St. '))

# city_saint_state = []
# for el in city_total_errors_list_state:
#     if el.find('Saint') != -1:
#         city_saint_state.append(el.replace('Saint ', 'St. '))

In [153]:
a = list(cities_q.keys())
a.sort(reverse=True)
cities_manual = []
print('Количество строк по этим городам = ', sum(a[:20]))
for el in a[:20]:
    cities_manual.extend(cities_q[el])

Количество строк по этим городам =  1890


Исправим часть ошибок вручную - это проще сделать, чем настраивать парсер для википедии в условиях ограниченного времени:

In [8]:
neighboors_1 = {'Flushing, NY': 'New York, NY', 'Jamaica, NY': 'New York, NY', 'Antioch, TN': 'Nashville, TN', 'Hixson, TN': 'Chattanooga, TN',
    'La Jolla, CA': 'San Diego, CA', 'North Hollywood, CA': 'Los Angeles, CA', 'Hermitage, TN': 'Nashville, TN', 'Inlet Beach, FL': 'Panama City, FL'
}
neighboors_2 = {'Kingwood, TX': 'Houston, TX', 'Huffman, TX': 'Houston, TX', 'Old Hickory, TN': 'Nashville, TN', 'Clearwater Beach, FL': 'Tampa, FL',
    'Forest Hills, NY': 'New York, NY', 'Madison, TN': 'Nashville, TN', 'Woodland Hills, CA': 'Los Angeles, CA', 'Lake In The Hills, IL': 'Chicago, IL',
    'Cordova, TN': 'Memphis, TN', 'Bayside, NY': 'New York, NY', 'Van Nuys, CA': 'Los Angeles, CA', 'Sylmar, CA': 'Los Angeles, CA', 'Astoria, NY':  'New York, NY',
    'College Point, NY': 'New York, NY', 'Whitestone, NY': 'New York, NY', 'East Elmhurst, NY': 'New York, NY', 'Corona, NY': 'New York, NY',
    'Long Island City, NY': 'New York, NY'
}

for k, v in neighboors_1.items():
    data_info.loc[data_info.city_2 == k, 'lat'] = data_info.loc[data_info.city_2 == v, 'lat'].iloc[0]
    data_info.loc[data_info.city_2 == k, 'lng'] = data_info.loc[data_info.city_2 == v, 'lng'].iloc[0]
    data_info.loc[data_info.city_2 == k, 'population'] = data_info.loc[data_info.city_2 == v, 'population'].iloc[0]
    data_info.loc[data_info.city_2 == k, 'density'] = data_info.loc[data_info.city_2 == v, 'density'].iloc[0]
    data_info.loc[data_info.city_2 == k, 'city_2'] = v
    print(k)

Flushing, NY
Jamaica, NY
Antioch, TN
Hixson, TN
La Jolla, CA
North Hollywood, CA
Hermitage, TN
Inlet Beach, FL


In [9]:
data_info.loc[data_info.city_2 == 'Huffman, TX', 'city_type'] = 'community'
data_info.loc[data_info.city_2 == 'Summerfield, FL', 'city_type'] = 'community'

data_info.loc[data_info.city_2 == 'Lakewood, NJ', 'lat'] = 40.0770
data_info.loc[data_info.city_2 == 'Lakewood, NJ', 'lng'] = -74.1985
data_info.loc[data_info.city_2 == 'Lakewood, NJ', 'population'] = 139506
data_info.loc[data_info.city_2 == 'Lakewood, NJ', 'density'] = 5476.2

data_info.loc[data_info.city_2 == 'Redford, MI', 'lat'] = 42.2341
data_info.loc[data_info.city_2 == 'Redford, MI', 'lng'] = -83.1749
data_info.loc[data_info.city_2 == 'Redford, MI', 'population'] = 49504
data_info.loc[data_info.city_2 == 'Redford, MI', 'density'] = 4302.7

data_info.loc[data_info.city_2 == 'Opa Locka, FL', 'lat'] = 25.5406
data_info.loc[data_info.city_2 == 'Opa Locka, FL', 'lng'] = -80.1503
data_info.loc[data_info.city_2 == 'Opa Locka, FL', 'population'] = 16463
data_info.loc[data_info.city_2 == 'Opa Locka, FL', 'density'] = 3821.5

data_info.loc[data_info.city_2 == 'Soddy Daisy, TN', 'lat'] = 35.1531
data_info.loc[data_info.city_2 == 'Soddy Daisy, TN', 'lng'] = -85.1037
data_info.loc[data_info.city_2 == 'Soddy Daisy, TN', 'population'] = 13070
data_info.loc[data_info.city_2 == 'Soddy Daisy, TN', 'density'] = 564

data_info.loc[data_info.city_2 == 'Saint Petersburg, FL', 'lat'] = 27.46
data_info.loc[data_info.city_2 == 'Saint Petersburg, FL', 'lng'] = -82.38
data_info.loc[data_info.city_2 == 'Saint Petersburg, FL', 'population'] = 261256
data_info.loc[data_info.city_2 == 'Saint Petersburg, FL', 'density'] = 4175.0

data_info.loc[data_info.city_2 == 'Mckinney, TX', 'lat'] = 33.1150
data_info.loc[data_info.city_2 == 'Mckinney, TX', 'lng'] = -96.3823
data_info.loc[data_info.city_2 == 'Mckinney, TX', 'population'] = 195308
data_info.loc[data_info.city_2 == 'Mckinney, TX', 'density'] = 2974.0

data_info.loc[data_info.city_2 == 'Port Saint Lucie, FL', 'lat'] = 27.1633
data_info.loc[data_info.city_2 == 'Port Saint Lucie, FL', 'lng'] = -80.2118
data_info.loc[data_info.city_2 == 'Port Saint Lucie, FL', 'population'] = 204851
data_info.loc[data_info.city_2 == 'Port Saint Lucie, FL', 'density'] = 1718.3

data_info.loc[data_info.city_2 == 'Saint Johns, FL', 'lat'] = 29.5775
data_info.loc[data_info.city_2 == 'Saint Johns, FL', 'lng'] = -81.3146
data_info.loc[data_info.city_2 == 'Saint Johns, FL', 'population'] = 86400
data_info.loc[data_info.city_2 == 'Saint Johns, FL', 'density'] = -999
data_info.loc[data_info.city_2 == 'Saint Johns, FL', 'city_type'] = 'community'

data_info.loc[data_info.city_2 == 'Winston Salem, NC', 'lat'] = 36.0610
data_info.loc[data_info.city_2 == 'Winston Salem, NC', 'lng'] = -80.1539
data_info.loc[data_info.city_2 == 'Winston Salem, NC', 'population'] = 250001
data_info.loc[data_info.city_2 == 'Winston Salem, NC', 'density'] = 1868.8

data_info.loc[data_info.city_2 == 'Mcallen, TX', 'lat'] = 26.1259
data_info.loc[data_info.city_2 == 'Mcallen, TX', 'lng'] = -98.1411
data_info.loc[data_info.city_2 == 'Mcallen, TX', 'population'] = 142210
data_info.loc[data_info.city_2 == 'Mcallen, TX', 'density'] = 2238

data_info.loc[data_info.city_2 == 'Rotonda West, FL', 'lat'] = 26.5316
data_info.loc[data_info.city_2 == 'Rotonda West, FL', 'lng'] = -82.1617
data_info.loc[data_info.city_2 == 'Rotonda West, FL', 'population'] = 10114
data_info.loc[data_info.city_2 == 'Rotonda West, FL', 'density'] = 949.05
data_info.loc[data_info.city_2 == 'Rotonda West, FL', 'city_type'] = 'community'

data_info.loc[data_info.city_2 == 'Placida, FL', 'lat'] = 26.4955
data_info.loc[data_info.city_2 == 'Placida, FL', 'lng'] = -82.1554
data_info.loc[data_info.city_2 == 'Placida, FL', 'population'] = -999
data_info.loc[data_info.city_2 == 'Placida, FL', 'density'] = -999
data_info.loc[data_info.city_2 == 'Placida, FL', 'city_type'] = 'community'

data_info.loc[data_info.city_2 == 'Saint Louis, MO', 'lat'] = 38.3738
data_info.loc[data_info.city_2 == 'Saint Louis, MO', 'lng'] = -90.1152
data_info.loc[data_info.city_2 == 'Saint Louis, MO', 'population'] = 293310
data_info.loc[data_info.city_2 == 'Saint Louis, MO', 'density'] = 4886.23

In [256]:
data_info.isna().sum().sort_values(ascending=False)

density       5483
population    3864
lat            369
lng            369
index            0
zipcode          0
state            0
city             0
city_state       0
city_type        0
dtype: int64

In [163]:
data_info = data_info.drop(['state_full_name', 'city+state'], axis=1)
data_info = data_info.rename(columns={'city_2': 'city_state'})

In [150]:
data_final = pd.read_csv('data/data_local_info.csv', dtype='object')

# исправим ошибки индексов
data_final = data_final[data_final.zipcode != '00000']
data_final = data_final[data_final.zipcode != '05642']
data_final = data_final[data_final.zipcode != '78697']

data_final.loc[data_final.zipcode == '90109', 'zipcode'] = '98109'
data_final.loc[data_final.zipcode == '98489', 'zipcode'] = '98498'
data_final.loc[data_final.zipcode == '33249', 'zipcode'] = '33149'
data_final.loc[data_final.zipcode == '22703', 'zipcode'] = '27703'
data_final.loc[data_final.zipcode == '98798', 'zipcode'] = '98198'

Обработаем полученные координаты. Возьмём из них координаты с наибольшей значимостью:

In [152]:
def coords_search(data):
    if len(data) > 2:
        number_of_coords = data.count('place_id')
        coords_list = []
        for c in range(number_of_coords):
            idx_b = data.find('\'lat\'')
            idx_e = data.find('display_name')
            coords = data[idx_b-1:idx_e-3]
            coords = re.sub(r'[^\d.,-]', '', coords)
            coords = coords.split(',')
            coords = [float(el) for el in coords]
            coords = tuple(coords)
            data = data[(idx_e + 5):]
            coords_list.append(coords)
        if len(coords_list) > 0:
            return coords_list[0]
        else:
            return np.NaN
    else:
        return np.NaN

data_final['coords'] = data_final['location'].map(coords_search)

Поправим адреса и попробуем ещё раз:

In [180]:
data_to_parse_new = data_final[data_final.coords.isna()]

# функция обрезает адрес до дом + улица
def street_correction(data):
    data = re.sub(r'St.+', 'St', data)
    data = re.sub(r'Street.+', 'St', data)
    data = re.sub(r'Rd.+', 'Rd', data)
    data = re.sub(r'Road.+', 'Road', data)
    data = re.sub(r'Ave.+', 'Ave', data)
    data = re.sub(r'Avenue.+', 'Avenue', data)
    data = re.sub(r'Lane.+', 'Lane', data)
    data = re.sub(r'Cir.+', 'Cir', data)
    data = re.sub(r'way.+', 'way', data)
    data = re.sub(r'Way.+', 'way', data)
    data = re.sub(r'Ter.+', 'Ter', data)
    data = re.sub(r'Ct.+', 'Ct', data)
    data = re.sub(r'Blvd.+', 'Blvd', data)
    data = re.sub(r'Boulevard.+', 'Blvd', data)
    data = re.sub(r'Sq.+', 'Sq', data)
    data = re.sub(r'Dr.+', 'Dr', data)
    data = re.sub(r'Drive.+', 'Drive', data)
    return data  

data_to_parse_new = data_to_parse_new.reset_index()
data_to_parse_new['street_new'] = data_to_parse_new['street'].map(street_correction)
data_to_parse_new.loc[keyword_mask(data_to_parse_new, 'street_new', ['isclosed']), 'street_new'] = 'no_address'
# data_to_parse_new.to_csv('data/data_to_parse_2.csv')

In [266]:
# data_info.to_csv('data/data_city_info.csv')
# data_final.to_csv('data/data_local_info.csv')